# imports

In [ ]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import cv2, sys
sys.path.append("/M3")
from M3 import m3F as m3F
from M3 import m3Face, m3tv
from M3 import m3HoughCircle as m3HoughCircle
from M3 import m3Batcher as m3Batcher
from M3 import m3Batch as m3Batch
from M3 import m3Pre as m3Pre
from M3 import m3Mask as m3Mask
from M3 import m3HoughCircle as m3HoughCircle
from M3 import m3MedianFilter, m3Denoising, retinexRunnable, m3Contrast
from M3 import m3Niels as m3Niels
from M3 import m3Class, m3Color, m3Size, m3Show, m3Edge, m3Comparison
#from M3 import m3ReduceAmountsOfColor




In [ ]:
# RUN ONCE, TO LOAD IMGS,  FIND EYES AND COMPARISONS MASKS

In [ ]:

# pre-stuff

preFunctions = {
    
         m3Face.findEyes68: {"show": True, "division": 8}, #returns eye.mask68
         m3Face.findEyes194: {"show": True, "division": 8}, #returns eye.194
         m3Batch.loadMasksForComparison: {"maskFolder": "PICTURES/five_mask/"}, #Test_set_easy_mask single_masked five_mask
    

}



pa = None

pa = m3Batcher.photoBatcher("PICTURES/five/", preFunctions) # five




print(pa.__dict__.items(), pa.photoArray)


#pa = m3Batch.photoBatch("PICTURES/single/", fArray, debug=True, debugIris=True, preArray=preFunctions, postArray=postFunctions)
# Test_set_easy single Test_set_hard

#pa.iterate(fArray)
#pa.post(postFunctions)



In [ ]:
# YOU CAN RUN THIS MULTIPLE TIMES AFTER THE FIRST HAS BEEN RUN, SO NOT TO LOAD/COMPUTE THE ABOVE

In [ ]:
fArray = [ { "doFor": {"original":"image", "doAs": "wip"}},
        {m3MedianFilter.medianFilter: {"kernelSize": 5, "show": False}},
        {retinexRunnable.runMSRCP: { "show": False}},
        {m3Color.reduceColor: {"n_colors": 5, "show": False}},
        {m3F.getRed: {"show": True}},
        {m3tv.denoise_tv_chambolle: {"weight": 1,"eps": 0.002,"n_iter_max": 200, "multichannel": True, "show": False}},
        {m3Color.reduceColor: {"n_colors": 5, "show": True}},
        {"doForEye": {}},
        {m3Mask.mask: { "img":"wip", "mask":"mask68", "dest": "noLids", "show": False}}, 
        {"doForEye": {}},
        {m3HoughCircle.findCircleAndMakeMask: { "eyeAttr":"noLids",  # returns a eye.houghMask
                                              "houghParams": 
                                              {"resolution": 1, "min_dist": 100, "param_1": 200, "param_2": 10, 
                                               "min_radius_width_divider": 7, "max_radius_width_divider": 5.5,
                                               "show": True},
                                              "maskParams" :
                                              {
                                                  "onlyOne": True
                                              }
                                             }},
            {"doForEye": {}},
            {m3Mask.mask: { "img":"houghMask", "mask":"mask194", "dest": "result", "show": False}},
           {"doFor": {"original":"result", "doAs": "result2"}},
            {m3MedianFilter.blur: {"kernelSize": (50,50), "show": True}},
         
          
         ]




postFunctions = {
                m3Comparison.pixelcomparison: {"eyeAttr": "result2"},
                m3Batch.generateComparison: {"outputName":
                                             "newdataset-fixedhohg", 
                                             "attrs": 
                                                    ["image", "houghOutline", "result2", "result", "testMask", "wip" ],
                                                     "folderName": "nielsline_TPFN_WBLUR5050", "exportFullMask": False, "CSV": True,
                                                     "settings": [preFunctions, fArray]
                            }, 
}

kMeanList = [2,3,4,5,6]
kMeanList = []
pa.iterate(fArray)
pa.post(postFunctions)

for index in range(0,(len(kMeanList))):
    m3F.printBlue("LOOPING" +  str(index))
    func = fArray[3]
#    print(params)
#    print("RCparams", RCparams)
    for f, p in func.items():
        p["n_colors"] = kMeanList[index]
  #  print("changed", p)
    GCParams = postFunctions[m3Batch.generateComparison]
    GCParams["folderName"] = "nielsFive_noBlur" + str(kMeanList[index])
  #  print(GCParams)
  #  GCParams = GCParams["foldername"]
 #   print( GCParams["foldername"])
    print("*******")
    print("fArray",fArray)
    pa.iterate(fArray)
    pa.post(postFunctions)
